# Extra ⭐

En este jupyter notebook se encuentra todo el proceso de exportación de datos para el estudio de las variables requeridas, utilizando la API Foursquare. **Para la empresa Boonty**.

*Consideraciones*
- Se ha establecido un "limit" de 10 elementos al realizar las llamadas a la API, para facilitar la selección, además de no superar el límite de llamadas a la API, permitida en la versión gratuita.
- La metodología empleada para la obtención de los datos ha sido la misma que seguida en los jupyter 2 y 3 para la empresa de LiveStream

## Índice 📎

1. Importación de librerías y funciones
2. Importación de los datos
3. Conexión con la API
4. Estudio de las variables
>- Aeropuertos
>- Discotecas
>- Starbucks
>- Restaurantes veganos
>- Estación de autobuses
>- Colegios
>- Veterinarios

## 1. Importación de librerías y funciones 📚

In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import geopandas as gpd
from pymongo import MongoClient,GEOSPHERE
from functools import reduce
import operator
from cartoframes.viz import Map, Layer, popup_element
from pymongo import MongoClient
import shapely.geometry
import math

Otras funciones:

In [2]:
#import Funciones Funciones as fu

## 2. Importación de los datos 📍

In [3]:
oficinas = pd.read_csv("./oficinas_info.csv")
oficinas

,Unnamed: 0,city,country_code,latitude,longitude,name,category_code,number_of_employees,founded_year
0,0,New York,USA,40.726155,-73.995625,Livestream,games_video,120,2007
1,1,New York,USA,40.717248,-74.002662,Boonty,games_video,150,2001


*Consideraciones: Se tomaran las coordenadas 40.717248, -74.002662 de la emprea Boonty para el estudio de las varibles*

## 3. Conexión con la API. Foursquare 🔌

In [4]:
url = "https://api.foursquare.com/v2/venues/explore"

In [5]:
client_id = open("./client_id.txt").readlines()[0].rstrip()

In [6]:
client_secret = open("./client_secret.txt").readlines()[0].rstrip()

In [7]:
coordenadas = "40.717248, -74.002662"

## 4. Estudio de las variables ✳️

## 4.1 Aeropuerto ✈️

In [8]:
params1 = {"client_id" : client_id,
              "client_secret" : client_secret,
              "v": "20180323",
              "ll": coordenadas,
              "query": "airport",
              "limit": 10,
              "radius": 5000
             }
response1 = requests.get(url,params=params1).json()

In [9]:
data1 = response1.get("response")

In [10]:
decode1 = data1.get("groups")[0] 

In [11]:
aeropuerto = decode1.get("items")

In [12]:
mapa_nombre = ["venue", "name"]
mapa_latitud = ["venue","location","lat"]
mapa_longitud = ["venue","location","lng"]

In [13]:
#Fuentes
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

In [14]:
#Extración de los datos
aeropuertos = []
for dic in aeropuerto:
    lista = {}
    lista["nombre"] = getFromDict(dic, mapa_nombre)
    lista["latitud"] = getFromDict(dic, mapa_latitud)
    lista["longitud"] = getFromDict(dic, mapa_longitud)
    aeropuertos.append(lista)

In [15]:
#Creación del DataFrame
df_aeropuertos = pd.DataFrame(aeropuertos)
df_aeropuertos

,nombre,latitud,longitud
0,Priority One Jets,40.709288,-74.006822
1,Corporate Express Travel,40.722429,-74.002404
2,PH Helipad,40.732918,-73.995468
3,Us Helicopter,40.702437,-74.013122
4,New Bell Car Service,40.693717,-73.960250
5,Atlantic Aviation,40.744048,-73.973925
6,New York City Limo Service,40.745192,-73.983371
7,New York Helicopter,40.701661,-74.009279
8,Mobil Car Service,40.711239,-73.951340
9,Times Square Limousine,40.752182,-74.003348


In [16]:
#Lo transformamos en GeoDataFrame
gdf_aeropuertos = gpd.GeoDataFrame(df_aeropuertos, geometry = gpd.points_from_xy(df_aeropuertos.longitud, df_aeropuertos.latitud))
gdf_aeropuertos

,nombre,latitud,longitud,geometry
0,Priority One Jets,40.709288,-74.006822,POINT (-74.00682 40.70929)
1,Corporate Express Travel,40.722429,-74.002404,POINT (-74.00240 40.72243)
2,PH Helipad,40.732918,-73.995468,POINT (-73.99547 40.73292)
3,Us Helicopter,40.702437,-74.013122,POINT (-74.01312 40.70244)
4,New Bell Car Service,40.693717,-73.960250,POINT (-73.96025 40.69372)
5,Atlantic Aviation,40.744048,-73.973925,POINT (-73.97393 40.74405)
6,New York City Limo Service,40.745192,-73.983371,POINT (-73.98337 40.74519)
7,New York Helicopter,40.701661,-74.009279,POINT (-74.00928 40.70166)
8,Mobil Car Service,40.711239,-73.951340,POINT (-73.95134 40.71124)
9,Times Square Limousine,40.752182,-74.003348,POINT (-74.00335 40.75218)


In [17]:
#Map(Layer(gdf_aeropuertos,"color:blue",popup_hover = [popup_element("nombre", "Aeropuertos")]))

🔴  **¿Cual será el más cercano?**

In [18]:
#Transformamos la información en tipo "point"
gdf_aeropuertos['geometry']=gdf_aeropuertos['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdf_aeropuertos

/usr/local/Caskroom/miniconda/base/lib/python3.8/site-packages/geopandas/geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,nombre,latitud,longitud,geometry
0,Priority One Jets,40.709288,-74.006822,"{'type': 'Point', 'coordinates': (-74.00682172..."
1,Corporate Express Travel,40.722429,-74.002404,"{'type': 'Point', 'coordinates': (-74.00240421..."
2,PH Helipad,40.732918,-73.995468,"{'type': 'Point', 'coordinates': (-73.99546813..."
3,Us Helicopter,40.702437,-74.013122,"{'type': 'Point', 'coordinates': (-74.01312247..."
4,New Bell Car Service,40.693717,-73.960250,"{'type': 'Point', 'coordinates': (-73.96025, 4..."
5,Atlantic Aviation,40.744048,-73.973925,"{'type': 'Point', 'coordinates': (-73.97392544..."
6,New York City Limo Service,40.745192,-73.983371,"{'type': 'Point', 'coordinates': (-73.98337125..."
7,New York Helicopter,40.701661,-74.009279,"{'type': 'Point', 'coordinates': (-74.00927889..."
8,Mobil Car Service,40.711239,-73.951340,"{'type': 'Point', 'coordinates': (-73.95133972..."
9,Times Square Limousine,40.752182,-74.003348,"{'type': 'Point', 'coordinates': (-74.00334835..."


In [19]:
client = MongoClient()
db = client["Ironhack-lab"]
collection11 = db.aeropuertos2
collection11.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [20]:
data11 = gdf_aeropuertos.to_dict(orient="records")
collection11.insert_many(data11)

In [21]:
cercano1 = db.aeropuertos2.find({"geometry":{"$near":{"type":"Point", "coordinates":[40.726155, -73.995625]}}})
list(cercano1)[0]

{'_id': ObjectId('60d9809b387729f83d1be9ee'),
 'nombre': 'New Bell Car Service',
 'latitud': 40.69371727,
 'longitud': -73.96025,
 'geometry': {'type': 'Point', 'coordinates': [-73.96025, 40.69371727]}}

In [22]:
def distancia(lat1, lon1, lat2, lon2):
    rad=math.pi/180
    dlat=lat2-lat1
    dlon=lon2-lon1
    R=6372.795477598
    a=(math.sin(rad*dlat/2))**2 + math.cos(rad*lat1)*math.cos(rad*lat2)*(math.sin(rad*dlon/2))**2
    distancia=2*R*math.asin(math.sqrt(a))
    return distancia

In [23]:
distancia(40.69371727,-73.96025,40.717248,-74.002662)

4.431503773419073

✅ Resumen: El más cercano es "New Bell Car Service" a 4,43 km.

## 4.2 Discotecas 💃

In [24]:
params2 = {"client_id" : client_id,
              "client_secret" : client_secret,
              "v": "20180323",
              "ll": coordenadas,
              "query": "nightclub",
              "limit": 10,
              "radius": 500
             }
response2 = requests.get(url,params=params2).json()

In [25]:
data2 = response2.get("response")
decode2 = data2.get("groups")[0] 
discoteca = decode2.get("items")

In [26]:
#Extración de los datos
discotecas = []
for dic in discoteca:
    lista = {}
    lista["nombre"] = getFromDict(dic, mapa_nombre)
    lista["latitud"] = getFromDict(dic, mapa_latitud)
    lista["longitud"] = getFromDict(dic, mapa_longitud)
    discotecas.append(lista)

In [27]:
#Creación del DataFrame
df_discotecas = pd.DataFrame(discotecas)
df_discotecas

,nombre,latitud,longitud
0,Paul's Cocktail Lounge,40.719563,-74.005035
1,The Blond,40.719242,-74.000066
2,The Studios At Tribeca Grand,40.719368,-74.004906
3,446 Loft,40.720421,-74.001227
4,M1-5 Lounge,40.719065,-74.003376


In [28]:
#Lo transformamos en GeoDataFrame
gdf_discotecas = gpd.GeoDataFrame(df_discotecas, geometry = gpd.points_from_xy(df_discotecas.longitud, df_discotecas.latitud))
gdf_discotecas

,nombre,latitud,longitud,geometry
0,Paul's Cocktail Lounge,40.719563,-74.005035,POINT (-74.00504 40.71956)
1,The Blond,40.719242,-74.000066,POINT (-74.00007 40.71924)
2,The Studios At Tribeca Grand,40.719368,-74.004906,POINT (-74.00491 40.71937)
3,446 Loft,40.720421,-74.001227,POINT (-74.00123 40.72042)
4,M1-5 Lounge,40.719065,-74.003376,POINT (-74.00338 40.71907)


In [29]:
#Map(Layer(gdf_discotecas,"color:pink",popup_hover = [popup_element("nombre", "Discotecas")]))

🔴 **¿Cual será la más cercano?** 

In [30]:
gdf_discotecas['geometry'] = gdf_discotecas['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdf_discotecas

/usr/local/Caskroom/miniconda/base/lib/python3.8/site-packages/geopandas/geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,nombre,latitud,longitud,geometry
0,Paul's Cocktail Lounge,40.719563,-74.005035,"{'type': 'Point', 'coordinates': (-74.00503511..."
1,The Blond,40.719242,-74.000066,"{'type': 'Point', 'coordinates': (-74.00006632..."
2,The Studios At Tribeca Grand,40.719368,-74.004906,"{'type': 'Point', 'coordinates': (-74.00490570..."
3,446 Loft,40.720421,-74.001227,"{'type': 'Point', 'coordinates': (-74.00122672..."
4,M1-5 Lounge,40.719065,-74.003376,"{'type': 'Point', 'coordinates': (-74.00337552..."


In [31]:
client = MongoClient()
db = client["Ironhack-lab"]
collection22 = db.discotecas2
collection22.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [32]:
data22 = gdf_discotecas.to_dict(orient="records")
collection22.insert_many(data22)

In [33]:
cercano2 = db.discotecas2.find({"geometry":{"$near":{"type":"Point", "coordinates":[40.726155, -73.995625],"$minDistance": 500, "$maxDistance":300}}})
list(cercano2)[0]

{'_id': ObjectId('60d980bf387729f83d1be9f6'),
 'nombre': 'The Blond',
 'latitud': 40.71924234753701,
 'longitud': -74.00006632090165,
 'geometry': {'type': 'Point',
  'coordinates': [-74.00006632090165, 40.71924234753701]}}

In [34]:
distancia(40.717248,-74.002662,40.71924234753701,-74.00006632090165)

0.31158894883856

✅ Resumen: La discoteca más cercana es "The Blond" a unos 311 metros.

## 4.3 Starbucks ☕

In [35]:
params3 = {"client_id" : client_id,
              "client_secret" : client_secret,
              "v": "20180323",
              "ll": coordenadas,
              "query": "starbucks",
              "limit": 10,
              "radius": 500
             }
response3 = requests.get(url,params=params3).json()

In [36]:
data3 = response3.get("response")
decode3 = data3.get("groups")[0] 
starbuck = decode3.get("items")

In [37]:
#Extración de los datos
starbucks = []
for dic in starbuck:
    lista = {}
    lista["nombre"] = getFromDict(dic, mapa_nombre)
    lista["latitud"] = getFromDict(dic, mapa_latitud)
    lista["longitud"] = getFromDict(dic, mapa_longitud)
    starbucks.append(lista)

In [38]:
df_starbucks = pd.DataFrame(starbucks)
df_starbucks

,nombre,latitud,longitud
0,Starbucks,40.720779,-74.004356
1,Starbucks,40.715714,-74.003154
2,Starbucks,40.719833,-73.999253
3,Starbucks,40.718230,-74.007320
4,Starbucks,40.718802,-74.002431
5,Starbucks,40.714855,-74.005936
6,Starbucks,40.718330,-74.000015
7,Starbucks,40.719852,-74.004950


In [39]:
#Lo transformamos en GeoDataFrame
gdf_starbucks = gpd.GeoDataFrame(df_starbucks, geometry = gpd.points_from_xy(df_starbucks.longitud, df_starbucks.latitud))
gdf_starbucks

,nombre,latitud,longitud,geometry
0,Starbucks,40.720779,-74.004356,POINT (-74.00436 40.72078)
1,Starbucks,40.715714,-74.003154,POINT (-74.00315 40.71571)
2,Starbucks,40.719833,-73.999253,POINT (-73.99925 40.71983)
3,Starbucks,40.718230,-74.007320,POINT (-74.00732 40.71823)
4,Starbucks,40.718802,-74.002431,POINT (-74.00243 40.71880)
5,Starbucks,40.714855,-74.005936,POINT (-74.00594 40.71486)
6,Starbucks,40.718330,-74.000015,POINT (-74.00002 40.71833)
7,Starbucks,40.719852,-74.004950,POINT (-74.00495 40.71985)


In [40]:
#Map(Layer(gdf_starbucks,"color:darkgreen",popup_hover = [popup_element("nombre", "Starbucks")]))

🔴 La empresa dispone de 5 Starbucks cercanos en un radio de 500 metros. Sin embargo de todos ellos:  
**¿Cual será el más cercano?** 

In [41]:
gdf_starbucks['geometry'] = gdf_starbucks['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdf_starbucks

/usr/local/Caskroom/miniconda/base/lib/python3.8/site-packages/geopandas/geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,nombre,latitud,longitud,geometry
0,Starbucks,40.720779,-74.004356,"{'type': 'Point', 'coordinates': (-74.00435638..."
1,Starbucks,40.715714,-74.003154,"{'type': 'Point', 'coordinates': (-74.00315416..."
2,Starbucks,40.719833,-73.999253,"{'type': 'Point', 'coordinates': (-73.99925263..."
3,Starbucks,40.718230,-74.007320,"{'type': 'Point', 'coordinates': (-74.00732, 4..."
4,Starbucks,40.718802,-74.002431,"{'type': 'Point', 'coordinates': (-74.00243121..."
5,Starbucks,40.714855,-74.005936,"{'type': 'Point', 'coordinates': (-74.005936, ..."
6,Starbucks,40.718330,-74.000015,"{'type': 'Point', 'coordinates': (-74.000015, ..."
7,Starbucks,40.719852,-74.004950,"{'type': 'Point', 'coordinates': (-74.00495, 4..."


In [42]:
client = MongoClient()
db = client["Ironhack-lab"]
collection33 = db.starbucks2
collection33.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [43]:
data33 = gdf_starbucks.to_dict(orient="records")
collection33.insert_many(data33)

In [44]:
cercano3 = db.starbucks2.find({"geometry":{"$near":{"type":"Point", "coordinates":[40.726155, -73.995625],"$minDistance": 500, "$maxDistance":300}}})
list(cercano3)[0]

{'_id': ObjectId('60d980de387729f83d1bea00'),
 'nombre': 'Starbucks',
 'latitud': 40.714855,
 'longitud': -74.005936,
 'geometry': {'type': 'Point', 'coordinates': [-74.005936, 40.714855]}}

In [45]:
distancia(40.717248,-74.002662,40.714855,-74.005936)

0.3834396671668515

✅ Resumen: El Starbucks más cercano se encuentra a 383 metros

## 4.4 Restaurante Vegano 🥕

In [46]:
params4 = {"client_id" : client_id,
              "client_secret" : client_secret,
              "v": "20180323",
              "ll": coordenadas,
              "query": "vegan restaurants",
              "limit": 10,
              "radius": 500
             }
response4 = requests.get(url,params=params4).json()

In [47]:
data4 = response4.get("response")
decode4 = data4.get("groups")[0] 
vegan = decode4.get("items")

In [48]:
#Extración de los datos
resveganos = []
for dic in vegan:
    lista = {}
    lista["nombre"] = getFromDict(dic, mapa_nombre)
    lista["latitud"] = getFromDict(dic, mapa_latitud)
    lista["longitud"] = getFromDict(dic, mapa_longitud)
    resveganos.append(lista)

In [49]:
#Creación del DataFrame
df_resveganos = pd.DataFrame(resveganos)
df_resveganos

,nombre,latitud,longitud
0,May Wah Vegetarian Market,40.718582,-73.999437
1,Le Botaniste,40.720545,-74.000138
2,Nish Nūsh,40.715537,-74.007725
3,Vegetarian Dim Sum House,40.714892,-73.998321
4,Happy Veggie Restaurant,40.716511,-73.997922
5,Galioto’s,40.718439,-73.997885


In [50]:
#Lo transformamos en GeoDataFrame
gdf_resveganos = gpd.GeoDataFrame(df_resveganos, geometry = gpd.points_from_xy(df_resveganos.longitud, df_resveganos.latitud))
gdf_resveganos

,nombre,latitud,longitud,geometry
0,May Wah Vegetarian Market,40.718582,-73.999437,POINT (-73.99944 40.71858)
1,Le Botaniste,40.720545,-74.000138,POINT (-74.00014 40.72054)
2,Nish Nūsh,40.715537,-74.007725,POINT (-74.00772 40.71554)
3,Vegetarian Dim Sum House,40.714892,-73.998321,POINT (-73.99832 40.71489)
4,Happy Veggie Restaurant,40.716511,-73.997922,POINT (-73.99792 40.71651)
5,Galioto’s,40.718439,-73.997885,POINT (-73.99788 40.71844)


In [51]:
#Map(Layer(gdf_resveganos,"color:green",popup_hover = [popup_element("nombre", "Restaurantes Veganos")]))

🔴 **¿Cual será el más cercano?** 

In [52]:
gdf_resveganos['geometry'] = gdf_resveganos['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdf_resveganos

/usr/local/Caskroom/miniconda/base/lib/python3.8/site-packages/geopandas/geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,nombre,latitud,longitud,geometry
0,May Wah Vegetarian Market,40.718582,-73.999437,"{'type': 'Point', 'coordinates': (-73.99943664..."
1,Le Botaniste,40.720545,-74.000138,"{'type': 'Point', 'coordinates': (-74.00013783..."
2,Nish Nūsh,40.715537,-74.007725,"{'type': 'Point', 'coordinates': (-74.00772452..."
3,Vegetarian Dim Sum House,40.714892,-73.998321,"{'type': 'Point', 'coordinates': (-73.99832147..."
4,Happy Veggie Restaurant,40.716511,-73.997922,"{'type': 'Point', 'coordinates': (-73.997922, ..."
5,Galioto’s,40.718439,-73.997885,"{'type': 'Point', 'coordinates': (-73.997885, ..."


In [53]:
client = MongoClient()
db = client["Ironhack-lab"]
collection44 = db.resveganos2
collection44.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [54]:
data44 = gdf_resveganos.to_dict(orient="records")
collection44.insert_many(data44)

In [55]:
cercano4 = db.resveganos2.find({"geometry":{"$near":{"type":"Point", "coordinates":[40.726155, -73.995625],"$minDistance": 500, "$maxDistance":300}}})
list(cercano4)[0]

{'_id': ObjectId('60d980fb387729f83d1bea07'),
 'nombre': 'Vegetarian Dim Sum House',
 'latitud': 40.71489232862965,
 'longitud': -73.99832147855244,
 'geometry': {'type': 'Point',
  'coordinates': [-73.99832147855244, 40.71489232862965]}}

In [56]:
distancia(40.717248,-74.002662,40.71489232862965,-73.99832147855244)

0.45005640400065644

✅ Resumen: El restaurante vegano más cercano se encuentra a 450 metros

## 4.5 Estación de autobuses 🚌

In [57]:
params5 = {"client_id" : client_id,
              "client_secret" : client_secret,
              "v": "20180323",
              "ll": coordenadas,
              "query": "bus station",
              "limit": 10,
              "radius": 500
             }
response5 = requests.get(url,params=params5).json()

In [58]:
data5 = response5.get("response")
decode5 = data5.get("groups")[0] 
bus = decode5.get("items")

In [59]:
#Extración de los datos
autobuses = []
for dic in bus:
    lista = {}
    lista["nombre"] = getFromDict(dic, mapa_nombre)
    lista["latitud"] = getFromDict(dic, mapa_latitud)
    lista["longitud"] = getFromDict(dic, mapa_longitud)
    autobuses.append(lista)

In [60]:
#Creación del DataFrame
df_autobuses = pd.DataFrame(autobuses)
df_autobuses

,nombre,latitud,longitud
0,MTA Bus - Broadway & White St (X1/X10/X12/X17),40.718120,-74.003042
1,NYU Shuttle - Route B,40.717025,-74.001726
2,x1 bus to SI,40.716549,-74.004366
3,Worth St & Broadway,40.716537,-74.005686
4,BookBuses,40.720591,-74.001031
5,MTA Bus - M22,40.713896,-73.999188
6,MTA - M5 - Church/Chambers,40.715057,-74.007607
7,NY Tour Bus Rental,40.715690,-74.006680
8,MTA - M9,40.713356,-74.001669
9,Canal St - Skyline Bus Tour,40.720917,-74.000688


In [61]:
#Lo transformamos en GeoDataFrame
gdf_autobuses = gpd.GeoDataFrame(df_autobuses, geometry = gpd.points_from_xy(df_autobuses.longitud, df_autobuses.latitud))
gdf_autobuses

,nombre,latitud,longitud,geometry
0,MTA Bus - Broadway & White St (X1/X10/X12/X17),40.718120,-74.003042,POINT (-74.00304 40.71812)
1,NYU Shuttle - Route B,40.717025,-74.001726,POINT (-74.00173 40.71703)
2,x1 bus to SI,40.716549,-74.004366,POINT (-74.00437 40.71655)
3,Worth St & Broadway,40.716537,-74.005686,POINT (-74.00569 40.71654)
4,BookBuses,40.720591,-74.001031,POINT (-74.00103 40.72059)
5,MTA Bus - M22,40.713896,-73.999188,POINT (-73.99919 40.71390)
6,MTA - M5 - Church/Chambers,40.715057,-74.007607,POINT (-74.00761 40.71506)
7,NY Tour Bus Rental,40.715690,-74.006680,POINT (-74.00668 40.71569)
8,MTA - M9,40.713356,-74.001669,POINT (-74.00167 40.71336)
9,Canal St - Skyline Bus Tour,40.720917,-74.000688,POINT (-74.00069 40.72092)


In [62]:
#Map(Layer(gdf_autobuses,"color:purple",popup_hover = [popup_element("nombre", "Estación de autobuses")]))

🔴 **¿Cual será el más cercano?** 

In [63]:
gdf_autobuses['geometry'] = gdf_autobuses['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdf_autobuses

/usr/local/Caskroom/miniconda/base/lib/python3.8/site-packages/geopandas/geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,nombre,latitud,longitud,geometry
0,MTA Bus - Broadway & White St (X1/X10/X12/X17),40.718120,-74.003042,"{'type': 'Point', 'coordinates': (-74.003042, ..."
1,NYU Shuttle - Route B,40.717025,-74.001726,"{'type': 'Point', 'coordinates': (-74.00172590..."
2,x1 bus to SI,40.716549,-74.004366,"{'type': 'Point', 'coordinates': (-74.00436571..."
3,Worth St & Broadway,40.716537,-74.005686,"{'type': 'Point', 'coordinates': (-74.00568556..."
4,BookBuses,40.720591,-74.001031,"{'type': 'Point', 'coordinates': (-74.00103092..."
5,MTA Bus - M22,40.713896,-73.999188,"{'type': 'Point', 'coordinates': (-73.999188, ..."
6,MTA - M5 - Church/Chambers,40.715057,-74.007607,"{'type': 'Point', 'coordinates': (-74.00760650..."
7,NY Tour Bus Rental,40.715690,-74.006680,"{'type': 'Point', 'coordinates': (-74.00668, 4..."
8,MTA - M9,40.713356,-74.001669,"{'type': 'Point', 'coordinates': (-74.00166859..."
9,Canal St - Skyline Bus Tour,40.720917,-74.000688,"{'type': 'Point', 'coordinates': (-74.0006876,..."


In [64]:
client = MongoClient()
db = client["Ironhack-lab"]
collection55 = db.autobuses2
collection55.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [65]:
data55 = gdf_autobuses.to_dict(orient="records")
collection55.insert_many(data55)

In [66]:
cercano5 = db.autobuses2.find({"geometry":{"$near":{"type":"Point", "coordinates":[40.726155, -73.995625],"$minDistance": 500, "$maxDistance":300}}})
list(cercano5)[0]

{'_id': ObjectId('60d9811a387729f83d1bea10'),
 'nombre': 'MTA Bus - M22',
 'latitud': 40.713896000000005,
 'longitud': -73.999188,
 'geometry': {'type': 'Point',
  'coordinates': [-73.999188, 40.713896000000005]}}

In [67]:
distancia(40.717248,-74.002662,40.713896000000005,-73.999188)

0.47410767949426486

✅ Resumen: La parada de autobuses más cercana se encuentra a 474 metros

## 4.6 Colegios 👶🏻

In [68]:
params6 = {"client_id" : client_id,
              "client_secret" : client_secret,
              "v": "20180323",
              "ll": coordenadas,
              "query": "school",
              "limit": 10,
              "radius": 500
             }
response6 = requests.get(url,params=params6).json() 

In [69]:
data6 = response6.get("response")
decode6 = data6.get("groups")[0] 
escuela = decode6.get("items")

In [70]:
#Extración de los datos
escuelas = []
for dic in escuela:
    lista = {}
    lista["nombre"] = getFromDict(dic, mapa_nombre)
    lista["latitud"] = getFromDict(dic, mapa_latitud)
    lista["longitud"] = getFromDict(dic, mapa_longitud)
    escuelas.append(lista)

In [71]:
#Creación del DataFrame
df_escuelas = pd.DataFrame(escuelas)
df_escuelas

,nombre,latitud,longitud
0,Gakko,40.718422,-73.999969
1,FasTracKids,40.718364,-73.998949
2,MUJI 無印良品 (MUJI),40.720709,-74.001015
3,New York Chinese School,40.716122,-73.998162
4,District 2 Pre-K Center,40.713982,-74.004059
5,PS 130,40.718857,-73.998581
6,Fine Jewelry School,40.715104,-74.006350
7,Greater Chinatown Community Association,40.714463,-73.999347
8,Cooking by the Book,40.717877,-74.007876
9,Reade Street Prep,40.715966,-74.008306


In [72]:
#Lo transformamos en GeoDataFrame
gdf_escuelas = gpd.GeoDataFrame(df_escuelas, geometry = gpd.points_from_xy(df_escuelas.longitud, df_escuelas.latitud))
gdf_escuelas

,nombre,latitud,longitud,geometry
0,Gakko,40.718422,-73.999969,POINT (-73.99997 40.71842)
1,FasTracKids,40.718364,-73.998949,POINT (-73.99895 40.71836)
2,MUJI 無印良品 (MUJI),40.720709,-74.001015,POINT (-74.00101 40.72071)
3,New York Chinese School,40.716122,-73.998162,POINT (-73.99816 40.71612)
4,District 2 Pre-K Center,40.713982,-74.004059,POINT (-74.00406 40.71398)
5,PS 130,40.718857,-73.998581,POINT (-73.99858 40.71886)
6,Fine Jewelry School,40.715104,-74.006350,POINT (-74.00635 40.71510)
7,Greater Chinatown Community Association,40.714463,-73.999347,POINT (-73.99935 40.71446)
8,Cooking by the Book,40.717877,-74.007876,POINT (-74.00788 40.71788)
9,Reade Street Prep,40.715966,-74.008306,POINT (-74.00831 40.71597)


In [73]:
#Map(Layer(gdf_escuelas,"color:orange",popup_hover = [popup_element("nombre", "Escuelas")]))

🔴  **¿Cual será el más cercano?** 

In [74]:
gdf_escuelas['geometry'] = gdf_escuelas['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdf_escuelas

/usr/local/Caskroom/miniconda/base/lib/python3.8/site-packages/geopandas/geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,nombre,latitud,longitud,geometry
0,Gakko,40.718422,-73.999969,"{'type': 'Point', 'coordinates': (-73.99996948..."
1,FasTracKids,40.718364,-73.998949,"{'type': 'Point', 'coordinates': (-73.998949, ..."
2,MUJI 無印良品 (MUJI),40.720709,-74.001015,"{'type': 'Point', 'coordinates': (-74.00101482..."
3,New York Chinese School,40.716122,-73.998162,"{'type': 'Point', 'coordinates': (-73.99816239..."
4,District 2 Pre-K Center,40.713982,-74.004059,"{'type': 'Point', 'coordinates': (-74.00405883..."
5,PS 130,40.718857,-73.998581,"{'type': 'Point', 'coordinates': (-73.99858131..."
6,Fine Jewelry School,40.715104,-74.006350,"{'type': 'Point', 'coordinates': (-74.00635, 4..."
7,Greater Chinatown Community Association,40.714463,-73.999347,"{'type': 'Point', 'coordinates': (-73.99934666..."
8,Cooking by the Book,40.717877,-74.007876,"{'type': 'Point', 'coordinates': (-74.00787643..."
9,Reade Street Prep,40.715966,-74.008306,"{'type': 'Point', 'coordinates': (-74.008306, ..."


In [75]:
client = MongoClient()
db = client["Ironhack-lab"]
collection66 = db.escuelas2
collection66.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [76]:
data66 = gdf_escuelas.to_dict(orient="records")
collection66.insert_many(data66)

In [77]:
cercano6 = db.escuelas2.find({"geometry":{"$near":{"type":"Point", "coordinates":[40.726155, -73.995625],"$minDistance": 500, "$maxDistance":300}}})
list(cercano6)[0]

{'_id': ObjectId('60d9813c387729f83d1bea1d'),
 'nombre': 'Greater Chinatown Community Association',
 'latitud': 40.714463390466406,
 'longitud': -73.99934666405763,
 'geometry': {'type': 'Point',
  'coordinates': [-73.99934666405763, 40.714463390466406]}}

In [78]:
distancia(40.717248,-74.002662,40.714463390466406,-73.99934666405763)

0.41718855922859505

✅ Resumen: El colegio más cercano se encuentra a 417 metros.

## 4.7 Veterinario 🐾

In [79]:
params7 = {"client_id" : client_id,
              "client_secret" : client_secret,
              "v": "20180323",
              "ll": coordenadas,
              "query": "veterinary",
              "limit": 10,
              "radius": 500
             }
response7 = requests.get(url,params=params7).json() 

In [80]:
data7 = response7.get("response")
decode7 = data7.get("groups")[0] 
vet = decode7.get("items")

In [81]:
#Extración de los datos
veterinario = []
for dic in vet:
    lista = {}
    lista["nombre"] = getFromDict(dic, mapa_nombre)
    lista["latitud"] = getFromDict(dic, mapa_latitud)
    lista["longitud"] = getFromDict(dic, mapa_longitud)
    veterinario.append(lista)

In [82]:
#Creación del DataFrame
df_veterinario = pd.DataFrame(veterinario)
df_veterinario

,nombre,latitud,longitud
0,Worth Street Veterinary Center,40.716753,-74.005673
1,PetCallVetNYC-Veterinary House Call Service,40.716999,-74.005836


In [83]:
#Lo transformamos en GeoDataFrame
gdf_veterinario = gpd.GeoDataFrame(df_veterinario, geometry = gpd.points_from_xy(df_veterinario.longitud, df_veterinario.latitud))
gdf_veterinario

,nombre,latitud,longitud,geometry
0,Worth Street Veterinary Center,40.716753,-74.005673,POINT (-74.00567 40.71675)
1,PetCallVetNYC-Veterinary House Call Service,40.716999,-74.005836,POINT (-74.00584 40.71700)


In [84]:
#Map(Layer(gdf_veterinario,"color:black",popup_hover = [popup_element("nombre", "Veterinarios")]))

🔴  **Distancia en metros** 

In [85]:
gdf_veterinario['geometry'] = gdf_veterinario['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdf_veterinario

/usr/local/Caskroom/miniconda/base/lib/python3.8/site-packages/geopandas/geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,nombre,latitud,longitud,geometry
0,Worth Street Veterinary Center,40.716753,-74.005673,"{'type': 'Point', 'coordinates': (-74.00567276..."
1,PetCallVetNYC-Veterinary House Call Service,40.716999,-74.005836,"{'type': 'Point', 'coordinates': (-74.00583648..."


In [86]:
client = MongoClient()
db = client["Ironhack-lab"]
collection77 = db.veterinario2
collection77.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [87]:
data77 = gdf_veterinario.to_dict(orient="records")
collection77.insert_many(data77)

In [88]:
cercano7 = db.veterinario2.find({"geometry":{"$near":{"type":"Point", "coordinates":[40.726155, -73.995625],"$minDistance": 500, "$maxDistance":300}}})
list(cercano7)[0]

{'_id': ObjectId('60d98159387729f83d1bea21'),
 'nombre': 'Worth Street Veterinary Center',
 'latitud': 40.71675266718981,
 'longitud': -74.0056727692407,
 'geometry': {'type': 'Point',
  'coordinates': [-74.0056727692407, 40.71675266718981]}}

In [89]:
distancia(40.717248,-74.002662,40.71675266718981,-74.0056727692407)

0.2597272505358649

✅ Resumen: El veterinario más cercano se encuentra a 259 metros.